In [157]:
import sys
import os
from dotenv import load_dotenv

root_dir = os.path.abspath("..")
sys.path.append(root_dir)
dotenv_path = os.path.join(root_dir, ".env")
load_dotenv(dotenv_path)

True

In [158]:
import polars as pl

In [159]:
FILE_FLIGHT_TRAIN = os.path.join(root_dir, "data", "v1", "processed_flight_features_train.parquet")
FILE_PRICE_TRAIN = os.path.join(root_dir, "data", "v1", "processed_pricing_features_train.parquet")
FILE_USER_TRAIN = os.path.join(root_dir, "data", "v1", "processed_user_features_train.parquet")
FILE_ADDITIONAL_TRAIN = os.path.join(root_dir, "data", "v1", "processed_additional_features_train.parquet")

In [160]:
FILE_FLIGHT_TEST = os.path.join(root_dir, "data", "v1", "processed_flight_features_test.parquet")
FILE_PRICE_TEST = os.path.join(root_dir, "data", "v1", "processed_pricing_features_test.parquet")
FILE_USER_TEST = os.path.join(root_dir, "data", "v1", "processed_user_features_test.parquet")
FILE_ADDITIONAL_TEST = os.path.join(root_dir, "data", "v1", "processed_additional_features_test.parquet")

In [161]:
print("[INFO] Reading TRAIN parquet files...")
train_flight = pl.read_parquet(FILE_FLIGHT_TRAIN)
train_price  = pl.read_parquet(FILE_PRICE_TRAIN)
train_user   = pl.read_parquet(FILE_USER_TRAIN)
train_add    = pl.read_parquet(FILE_ADDITIONAL_TRAIN)

print("[INFO] Shapes:")
print("flight:", train_flight.shape)
print("price :", train_price.shape)
print("user  :", train_user.shape)
print("add   :", train_add.shape)

print("[INFO] Reading TEST parquet files...")
test_flight = pl.read_parquet(FILE_FLIGHT_TEST)
test_price  = pl.read_parquet(FILE_PRICE_TEST)
test_user   = pl.read_parquet(FILE_USER_TEST)
test_add    = pl.read_parquet(FILE_ADDITIONAL_TEST)

print("[INFO] Shapes:")
print("flight:", test_flight.shape)
print("price :", test_price.shape)
print("user  :", test_user.shape)
print("add   :", test_add.shape)

[INFO] Reading TRAIN parquet files...
[INFO] Shapes:
flight: (18145372, 53)
price : (18145372, 10)
user  : (18145372, 10)
add   : (18145372, 8)
[INFO] Reading TEST parquet files...
[INFO] Shapes:
flight: (6897776, 53)
price : (6897776, 10)
user  : (6897776, 10)
add   : (6897776, 8)


In [162]:
train_df = (
    train_flight
    .join(train_price, on="row_id")
    .join(train_user, on="row_id")
    .join(train_add, on="row_id")
)

test_df = (
    test_flight
    .join(test_price, on="row_id")
    .join(test_user, on="row_id")
    .join(test_add, on="row_id")
)

### Add ranker_id and selected

In [163]:
train_org_file = os.path.join(root_dir, "kaggle", "train.parquet")
test_org_file = os.path.join(root_dir, "kaggle", "test.parquet")

In [164]:
train_org = pl.scan_parquet(train_org_file)
test_org = pl.scan_parquet(test_org_file)

train_org_group_id = train_org.select('ranker_id').with_row_index('row_id')
test_org_group_id = test_org.select('ranker_id').with_row_index('row_id')

train_target = train_org.select('selected').with_row_index('row_id')

In [165]:
train_df = (
    train_df
    .join(train_org_group_id.collect(), on='row_id')
    .sort(["ranker_id", "go_total_flight_time"])
    .with_columns([
        pl.col("go_total_flight_time").rank(method='dense').over('ranker_id').alias('go_total_flight_time_rank'),
        (pl.col("go_total_flight_time") - pl.col("go_total_flight_time").min().over('ranker_id')).fill_null(0.0).alias('go_total_flight_time_delta')
    ])
    .sort('row_id')
    .sort(["ranker_id", "rtn_total_flight_time"])
    .with_columns([
        pl.col("rtn_total_flight_time").rank(method='dense').over('ranker_id').alias('rtn_total_flight_time_rank'),
        (pl.col("rtn_total_flight_time") - pl.col("rtn_total_flight_time").min().over('ranker_id')).fill_null(0.0).alias('rtn_total_flight_time_delta')
    ])
    .sort('row_id')
)
test_df = (
    test_df
    .join(test_org_group_id.collect(), on='row_id')
    .sort(["ranker_id", "go_total_flight_time"])
    .with_columns([
        pl.col("go_total_flight_time").rank(method='dense').over('ranker_id').alias('go_total_flight_time_rank'),
        (pl.col("go_total_flight_time") - pl.col("go_total_flight_time").min().over('ranker_id')).fill_null(0.0).alias('go_total_flight_time_delta')
    ])
    .sort('row_id')
    .sort(["ranker_id", "rtn_total_flight_time"])
    .with_columns([
        pl.col("rtn_total_flight_time").rank(method='dense').over('ranker_id').alias('rtn_total_flight_time_rank'),
        (pl.col("rtn_total_flight_time") - pl.col("rtn_total_flight_time").min().over('ranker_id')).fill_null(0.0).alias('rtn_total_flight_time_delta')
    ])
    .sort('row_id')
)

# Last Engineering

### Dropped columns and why

#### 1. Travel distance

```python
train_df.select([
    pl.corr("go_travel_distance", "go_total_flight_time")
])

# 0.97xxx
```

Verdict: Drop `go_travel_distance`, `rtn_travel_distance`

In [166]:
def features_seg_number(df: pl.DataFrame) -> pl.DataFrame:
    # Start with row_id to preserve identity. 
    base = df.select(['row_id', 'go_seg_number', 'rtn_seg_number'])

    base = base.with_columns([
        (pl.col("rtn_seg_number") == 0).cast(pl.Int8).alias("is_one_way")
    ])
    return base

def features_flight_time(df: pl.DataFrame) -> pl.DataFrame:
    enriched = (
        df
        .select([
            "row_id",
            "go_total_flight_time",
            "rtn_total_flight_time",
            'ranker_id'
        ])
        .with_columns([
            pl.col("go_total_flight_time").rank(method='ordinal').over('ranker_id').alias('go_total_flight_time_rank'),
            pl.col("rtn_total_flight_time").rank(method='ordinal').over('ranker_id').alias('rtn_total_flight_time_rank'),
            (pl.col("go_total_flight_time") - pl.col("go_total_flight_time").min().over('ranker_id')).alias('go_total_flight_time_delta'),
            (pl.col("rtn_total_flight_time") - pl.col("rtn_total_flight_time").min().over('ranker_id')).alias('rtn_total_flight_time_delta'),
        ])
        .select([
            'row_id', 
            'go_total_flight_time_rank', 
            'rtn_total_flight_time_rank', 
            'go_total_flight_time_delta', 
            'rtn_total_flight_time_delta'
        ])
    )

    return enriched

def features_bag_allow_0(df: pl.DataFrame) -> pl.DataFrame:
    return (
        df
        .select([
            "row_id",
            "go_bag_allow_0",
            "rtn_bag_allow_0",
        ])
        .with_columns([
            pl.col("go_bag_allow_0").fill_null(0).alias("bag0_go"),
            pl.col("rtn_bag_allow_0").fill_null(0).alias("bag0_rtn")
        ])
        .with_columns([
            # Binary flags
            (pl.col("bag0_go") > 0).cast(pl.Int8).alias("go_has_bag_allow"),
            (pl.col("bag0_rtn") > 0).cast(pl.Int8).alias("rtn_has_bag_allow")
        ])
        .select([
            "row_id",
            "bag0_go",
            "bag0_rtn",
            "go_has_bag_allow",
            "rtn_has_bag_allow"
        ])
    )

def features_plane_comfort_score(df: pl.DataFrame) -> pl.DataFrame:
    df_filled = df.select([
        pl.col("row_id"),
        pl.col("ranker_id"),
        pl.col("go_plane_size_0").fill_null(0.0),
        pl.col("go_plane_size_1").fill_null(0.0),
        pl.col("go_plane_size_2").fill_null(0.0),
        pl.col("go_flight_time_0").fill_null(0.0),
        pl.col("go_flight_time_1").fill_null(0.0),
        pl.col("go_flight_time_2").fill_null(0.0),
        pl.col("rtn_plane_size_0").fill_null(0.0),
        pl.col("rtn_plane_size_1").fill_null(0.0),
        pl.col("rtn_plane_size_2").fill_null(0.0),
        pl.col("rtn_flight_time_0").fill_null(0.0),
        pl.col("rtn_flight_time_1").fill_null(0.0),
        pl.col("rtn_flight_time_2").fill_null(0.0),
    ])

    df_filled = (
        df_filled
        .with_columns([
            # Numerator: sum(flight_time * plane_size)
            (
                pl.col("go_flight_time_0") * pl.col("go_plane_size_0") +
                pl.col("go_flight_time_1") * pl.col("go_plane_size_1") +
                pl.col("go_flight_time_2") * pl.col("go_plane_size_2")
            ).alias("plane_size_weighted_sum_go"),
            (
                pl.col("rtn_flight_time_0") * pl.col("rtn_plane_size_0") +
                pl.col("rtn_flight_time_1") * pl.col("rtn_plane_size_1") +
                pl.col("rtn_flight_time_2") * pl.col("rtn_plane_size_2")
            ).alias("plane_size_weighted_sum_rtn"),

            # Denominator: sum(flight_time)
            (
                pl.col("go_flight_time_0") + pl.col("go_flight_time_1") + pl.col("go_flight_time_2")
            ).alias("plane_size_weighted_sum_go_denom"),
            (
                pl.col("rtn_flight_time_0") + pl.col("rtn_flight_time_1") + pl.col("rtn_flight_time_2")
            ).alias("plane_size_weighted_sum_rtn_denom"),
        ])
        .with_columns([
            pl.when(pl.col("plane_size_weighted_sum_go_denom") > 0)
            .then(pl.col("plane_size_weighted_sum_go") / pl.col("plane_size_weighted_sum_go_denom"))
            .otherwise(0.0)
            .alias("go_plane_comfort_score"),
            pl.when(pl.col("plane_size_weighted_sum_rtn_denom") > 0)
            .then(pl.col("plane_size_weighted_sum_rtn") / pl.col("plane_size_weighted_sum_rtn_denom"))
            .otherwise(0.0)
            .alias("rtn_plane_comfort_score"),
        ])
        .with_columns([
            # Groupwise rank and delta
            pl.col("go_plane_comfort_score").rank("ordinal").over("ranker_id").alias("go_plane_comfort_score_rank"),
            pl.col("rtn_plane_comfort_score").rank("ordinal").over("ranker_id").alias("rtn_plane_comfort_score_rank"),
            (pl.col("go_plane_comfort_score") - pl.col("go_plane_comfort_score").min().over("ranker_id")).alias("go_plane_comfort_score_delta"),
            (pl.col("rtn_plane_comfort_score") - pl.col("rtn_plane_comfort_score").min().over("ranker_id")).alias("rtn_plane_comfort_score_delta"),
        ])
    )

    return df_filled.select([
        "row_id",
        "go_plane_comfort_score",
        "rtn_plane_comfort_score",
        "go_plane_comfort_score_rank",
        "rtn_plane_comfort_score_rank",
        "go_plane_comfort_score_delta",
        "rtn_plane_comfort_score_delta"
    ])

def features_airline_service_score(df: pl.DataFrame) -> pl.DataFrame:
    """
    Score ≈ 0.0 → All segments are major carriers
	Score ≈ 1.0 → All segments are low-cost carriers
	Score ≈ 0.5 → Likely a mix
    """
    return (
        df.select([
            "row_id", "ranker_id",
            "go_airline_is_lcc_0", "go_airline_is_lcc_1", "go_airline_is_lcc_2",
            "go_flight_time_0", "go_flight_time_1", "go_flight_time_2",
            "rtn_airline_is_lcc_0", "rtn_airline_is_lcc_1", "rtn_airline_is_lcc_2",
            "rtn_flight_time_0", "rtn_flight_time_1", "rtn_flight_time_2",
        ])
        .with_columns([
            # Fill nulls
            pl.col("go_airline_is_lcc_0").fill_null(1),
            pl.col("go_airline_is_lcc_1").fill_null(1),
            pl.col("go_airline_is_lcc_2").fill_null(1),
            pl.col("rtn_airline_is_lcc_0").fill_null(1),
            pl.col("rtn_airline_is_lcc_1").fill_null(1),
            pl.col("rtn_airline_is_lcc_2").fill_null(1),
        ])
        .with_columns([
            # GO weighted LCC score
            (
                pl.sum_horizontal([
                    pl.col("go_airline_is_lcc_0").fill_null(1) * pl.col("go_flight_time_0").fill_null(0),
                    pl.col("go_airline_is_lcc_1").fill_null(1) * pl.col("go_flight_time_1").fill_null(0),
                    pl.col("go_airline_is_lcc_2").fill_null(1) * pl.col("go_flight_time_2").fill_null(0),
                ]) / (
                    pl.col("go_flight_time_0").fill_null(0) + pl.col("go_flight_time_1").fill_null(0) + pl.col("go_flight_time_2").fill_null(0) + 1e-6
                )
            ).alias("go_airline_service_score"),

            # RTN weighted LCC score
            (
                pl.sum_horizontal([
                    pl.col("rtn_airline_is_lcc_0").fill_null(1) * pl.col("rtn_flight_time_0").fill_null(0),
                    pl.col("rtn_airline_is_lcc_1").fill_null(1) * pl.col("rtn_flight_time_1").fill_null(0),
                    pl.col("rtn_airline_is_lcc_2").fill_null(1) * pl.col("rtn_flight_time_2").fill_null(0),
                ]) / (
                    pl.col("rtn_flight_time_0").fill_null(0) + pl.col("rtn_flight_time_1").fill_null(0) + pl.col("rtn_flight_time_2").fill_null(0) + 1e-6
                )
            ).alias("rtn_airline_service_score"),
        ])
        .with_columns([
            # Groupwise rank and delta
            pl.col("go_airline_service_score").rank("ordinal").over("ranker_id").alias("go_airline_service_score_rank"),
            pl.col("rtn_airline_service_score").rank("ordinal").over("ranker_id").alias("rtn_airline_service_score_rank"),
            (pl.col("go_airline_service_score") - pl.col("go_airline_service_score").min().over("ranker_id")).alias("go_airline_service_score_delta"),
            (pl.col("rtn_airline_service_score") - pl.col("rtn_airline_service_score").min().over("ranker_id")).alias("rtn_airline_service_score_delta"),
        ])
        .select([
            "row_id",
            "go_airline_service_score",
            "rtn_airline_service_score",
            "go_airline_service_score_rank",
            "rtn_airline_service_score_rank",
            "go_airline_service_score_delta",
            "rtn_airline_service_score_delta"
        ])

    )

def features_international(df: pl.DataFrame) -> pl.DataFrame:
    return (
        df
        .select([
            "row_id",
            "go_international",
            "rtn_international"
        ])
        .with_columns([
            (pl.col("go_international")).cast(pl.Int8).alias("go_international"),
            (pl.col("rtn_international")).cast(pl.Int8).alias("rtn_international")
        ])
        .select([
            "row_id",
            "go_international",
            "rtn_international"
        ])
    )

def features_seat_comfort_score(df: pl.DataFrame) -> pl.DataFrame:
    df_filled = df.select([
        pl.col("row_id"),
        pl.col("ranker_id"),
        pl.col("go_seat_types_0").fill_null(0.0),
        pl.col("go_seat_types_1").fill_null(0.0),
        pl.col("go_seat_types_2").fill_null(0.0),
        pl.col("go_flight_time_0").fill_null(0.0),
        pl.col("go_flight_time_1").fill_null(0.0),
        pl.col("go_flight_time_2").fill_null(0.0),
        pl.col("rtn_seat_types_0").fill_null(0.0),
        pl.col("rtn_seat_types_1").fill_null(0.0),
        pl.col("rtn_seat_types_2").fill_null(0.0),
        pl.col("rtn_flight_time_0").fill_null(0.0),
        pl.col("rtn_flight_time_1").fill_null(0.0),
        pl.col("rtn_flight_time_2").fill_null(0.0),
    ])

    df_filled = (
        df_filled
        .with_columns([
            # Numerator: sum(flight_time * plane_size)
            (
                pl.col("go_flight_time_0") * pl.col("go_seat_types_0") +
                pl.col("go_flight_time_1") * pl.col("go_seat_types_1") +
                pl.col("go_flight_time_2") * pl.col("go_seat_types_2")
            ).alias("seat_types_weighted_sum_go"),
            (
                pl.col("rtn_flight_time_0") * pl.col("rtn_seat_types_0") +
                pl.col("rtn_flight_time_1") * pl.col("rtn_seat_types_1") +
                pl.col("rtn_flight_time_2") * pl.col("rtn_seat_types_2")
            ).alias("seat_types_weighted_sum_rtn"),

            # Denominator: sum(flight_time)
            (
                pl.col("go_flight_time_0") + pl.col("go_flight_time_1") + pl.col("go_flight_time_2")
            ).alias("seat_types_weighted_sum_go_denom"),
            (
                pl.col("rtn_flight_time_0") + pl.col("rtn_flight_time_1") + pl.col("rtn_flight_time_2")
            ).alias("seat_types_weighted_sum_rtn_denom"),
        ])
        .with_columns([
            pl.when(pl.col("seat_types_weighted_sum_go_denom") > 0)
            .then(pl.col("seat_types_weighted_sum_go") / pl.col("seat_types_weighted_sum_go_denom"))
            .otherwise(0.0)
            .alias("go_seat_score"),
            pl.when(pl.col("seat_types_weighted_sum_rtn_denom") > 0)
            .then(pl.col("seat_types_weighted_sum_rtn") / pl.col("seat_types_weighted_sum_rtn_denom"))
            .otherwise(0.0)
            .alias("rtn_seat_score"),
        ])
        .with_columns([
            # Groupwise rank and delta
            pl.col("go_seat_score").rank("ordinal").over("ranker_id").alias("go_seat_score_rank"),
            pl.col("rtn_seat_score").rank("ordinal").over("ranker_id").alias("rtn_seat_score_rank"),
            (pl.col("go_seat_score") - pl.col("go_seat_score").min().over("ranker_id")).alias("go_seat_score_delta"),
            (pl.col("rtn_seat_score") - pl.col("rtn_seat_score").min().over("ranker_id")).alias("rtn_seat_score_delta"),
        ])
    )

    return df_filled.select([
        "row_id",
        "go_seat_score",
        "rtn_seat_score",
        "go_seat_score_rank",
        "rtn_seat_score_rank",
        "go_seat_score_delta",
        "rtn_seat_score_delta"
    ])

def features_pure_rank(df: pl.DataFrame) -> pl.DataFrame:
    return (
        df
        .select([
            "row_id",
            "total_price",
            "taxes",
            "tax_ratio",
            "mini_rule0_penalty",
            "mini_rule0_pct",
            "mini_rule1_penalty",
            "mini_rule1_pct",
            "count_segments_in_ff",
            "departure_rank",
            "ranker_id"
        ])
        .with_columns([
            # Groupwise ranking
            pl.col(c).rank("ordinal").over("ranker_id").alias(f"{c}_rank")
            for c in [
                "total_price", "taxes", "tax_ratio",
                "mini_rule0_penalty", "mini_rule0_pct",
                "mini_rule1_penalty", "mini_rule1_pct",
                "count_segments_in_ff",
                "departure_rank",
            ]
        ] + [
            # Groupwise delta from min
            (pl.col(c) - pl.col(c).min().over("ranker_id")).alias(f"{c}_delta")
            for c in [
                "total_price", "taxes", "tax_ratio",
                "mini_rule0_penalty", "mini_rule0_pct",
                "mini_rule1_penalty", "mini_rule1_pct",
                "count_segments_in_ff",
                "departure_rank",
            ]
        ])
        .select(["row_id"] + [
            f"{c}_{suffix}"
            for c in [
                "total_price", "taxes", "tax_ratio",
                "mini_rule0_penalty", "mini_rule0_pct",
                "mini_rule1_penalty", "mini_rule1_pct",
                "count_segments_in_ff",
                "departure_rank",
            ]
            for suffix in ["rank", "delta"]
        ] + [
            "total_price", "taxes", "tax_ratio",
            "mini_rule0_penalty", "mini_rule0_pct",
            "mini_rule1_penalty", "mini_rule1_pct",
            "count_segments_in_ff",
            "departure_rank",
        ])
    )

def features_no_edit(df: pl.DataFrame) -> pl.DataFrame:
    return (
        df
        .select([
            "row_id",
            "is_access_tp",
            "is_self_booking",
            "is_vip",
            "is_male",
            "is_access_3d",
            "has_corporate_tariff",
            "is_preferred_airline",
            "passenger_count",
        ])
    )

In [167]:
tr1 = features_seg_number(train_df)
tr2 = features_flight_time(train_df)
tr3 = features_bag_allow_0(train_df)
tr4 = features_plane_comfort_score(train_df)
tr5 = features_airline_service_score(train_df)
tr6 = features_international(train_df)
tr7 = features_seat_comfort_score(train_df)
tr8 = features_pure_rank(train_df)
tr9 = features_no_edit(train_df)

train_df2 = (
    tr1
    .join(tr2, on="row_id")
    .join(tr3, on="row_id")
    .join(tr4, on="row_id")
    .join(tr5, on="row_id")
    .join(tr6, on="row_id")
    .join(tr7, on="row_id")
    .join(tr8, on="row_id")
    .join(tr9, on="row_id")
)

te1 = features_seg_number(test_df)
te2 = features_flight_time(test_df)
te3 = features_bag_allow_0(test_df)
te4 = features_plane_comfort_score(test_df)
te5 = features_airline_service_score(test_df)
te6 = features_international(test_df)
te7 = features_seat_comfort_score(test_df)
te8 = features_pure_rank(test_df)
te9 = features_no_edit(test_df)

test_df2 = (
    te1
    .join(te2, on="row_id")
    .join(te3, on="row_id")
    .join(te4, on="row_id")
    .join(te5, on="row_id")
    .join(te6, on="row_id")
    .join(te7, on="row_id")
    .join(te8, on="row_id")
    .join(te9, on="row_id")
)

In [168]:
print("train_df.shape", train_df2.shape)
print("train_target.shape", train_target.collect().shape)
print("")
print("test_df.shape", test_df.shape)

train_df.shape (18145372, 67)
train_target.shape (18145372, 2)

test_df.shape (6897776, 83)


In [169]:
train_df2.write_parquet(os.path.join(root_dir, "data", "v2", "train.parquet"))
train_target.collect().write_parquet(os.path.join(root_dir, "data", "v2", "train_target.parquet"))
test_df2.write_parquet(os.path.join(root_dir, "data", "v2", "test.parquet"))